Built In Algorithms

Setup

In [2]:
%cd /root/sagemaker/notebooks/

/root/sagemaker/notebooks


In [3]:
import boto3
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)

BUCKET = 'sagemaker-churn-20220310'
PREFIX = 'churn'
LOCAL_DATA_DIRECTORY = f'../data/{PREFIX}'

print(f"Artifacts will be in written to s3://{BUCKET}/{PREFIX}")

Artifacts will be in written to s3://sagemaker-churn-20220310/churn


In [4]:
import sagemaker

In [5]:
sagemaker_session = sagemaker.Session()
boto_session = sagemaker_session.boto_session

In [7]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

arn:aws:iam::104877823522:role/service-role/AmazonSageMaker-ExecutionRole-20220310T230092


Data

In [8]:
local_data_path = f'{LOCAL_DATA_DIRECTORY}/training-dataset-with-header.csv'
data = pd.read_csv(local_data_path)
data

,Churn,Account Length,VMail Message,Day Mins,Day Calls,Eve Mins,Eve Calls,Night Mins,Night Calls,Intl Mins,Intl Calls,CustServ Calls,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,0,106,0,274.4,120,198.6,82,160.8,62,6.0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
1,0,28,0,187.8,94,248.6,86,208.8,124,10.6,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0
2,1,148,0,279.3,104,201.6,87,280.8,99,7.9,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0
3,0,132,0,191.9,107,206.9,127,272.0,88,12.6,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
4,0,92,29,155.4,110,188.5,104,254.9,118,8.0,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,0,106,0,194.8,133,213.4,73,190.8,92,11.5,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0
2329,1,125,0,143.2,80,88.1,94,233.2,135,8.8,7,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2330,0,129,0,143.7,114,297.8,98,212.6,86,11.4,8,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0
2331,0,159,0,198.8,107,195.5,91,213.3,120,16.5,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [11]:
s3_input_train = sagemaker_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/train.csv',
                                              bucket = BUCKET,
                                              key_prefix = PREFIX)

s3_input_validation = sagemaker_session.upload_data(f'{LOCAL_DATA_DIRECTORY}/validation.csv',
                                                   bucket = BUCKET,
                                                   key_prefix = PREFIX)

s3_input_train = sagemaker.TrainingInput(s3_data=s3_input_train, content_type='csv')
s3_input_validation = sagemaker.TrainingInput(s3_data=s3_input_validation, content_type='csv')


Training

In [12]:
from sagemaker import image_uris

xgboost_image_uri = image_uris.retrieve(framework='xgboost', region=boto_session.region_name, version='0.90-2')
xgboost_image_uri

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-2-cpu-py3'

In [17]:
from sagemaker import estimator

xgb_model = estimator.Estimator(image_uri=xgboost_image_uri,
                              role=role,
                              instance_count=1,
                              instance_type='ml.m4.xlarge',
                              output_path=f's3://{BUCKET}/{PREFIX}',
                              base_job_name='builtin-xgboost',
                              sagemaker_session=sagemaker_session)

In [18]:
xgb_model.set_hyperparameters(max_depth=5,
                             subsample=0.8,
                             num_round=600,
                             eta=0.2,
                             gamma=4,
                             min_child_weight=6,
                             silent=0,
                             objective='binary:logistic')

In [19]:
xgb_model.fit({'train':s3_input_train, 'validation':s3_input_validation})

2022-03-11 18:31:25 Starting - Starting the training job...
2022-03-11 18:31:48 Starting - Launching requested ML instancesProfilerReport-1647023485: InProgress
.........
2022-03-11 18:33:08 Starting - Preparing the instances for training......
2022-03-11 18:34:23 Downloading - Downloading input data......
2022-03-11 18:35:25 Training - Downloading the training image...
2022-03-11 18:35:51 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[18:35:55] 2333x69 m

Custom script to train model